# Working with JSON files in Python

Working with JSON files isn't the most fun. While pandas has the read_json method that is useful for reading the .json file into a dataframe, we are often left with lists or dictionaries inside of columns. Since nested column values aren't really helpful for analzying data, we'll explore some methods for unpacking the json and creating clean and orderly dataframes.

In [6]:
import numpy as np
import pandas as pd
import ijson
from pandas.io.json import json_normalize

In [10]:
%%bash
# we can use %%bash magic to print a preview of our file

head ../ETL/roam_prescription_based_prediction.jsonl

{"cms_prescription_counts": {"DOXAZOSIN MESYLATE": 26, "MIDODRINE HCL": 12, "MEGESTROL ACETATE": 11, "BENAZEPRIL HCL": 11, "METOLAZONE": 73, "NOVOLOG": 12, "DIAZEPAM": 24, "HYDRALAZINE HCL": 50, "SENSIPAR": 94, "LABETALOL HCL": 28, "PREDNISONE": 40, "CALCITRIOL": 79, "HYDROCODONE-ACETAMINOPHEN": 64, "HYDROCHLOROTHIAZIDE": 59, "LOSARTAN-HYDROCHLOROTHIAZIDE": 14, "FENOFIBRATE": 14, "MINOXIDIL": 14, "MELOXICAM": 29, "ATENOLOL": 62, "CARISOPRODOL": 40, "GABAPENTIN": 35, "OMEPRAZOLE": 35, "KLOR-CON M10": 20, "LANTUS": 20, "AMLODIPINE BESYLATE": 175, "CARVEDILOL": 36, "LOSARTAN POTASSIUM": 41, "IRBESARTAN": 11, "NIFEDICAL XL": 32, "NIFEDIPINE ER": 51, "LEVOTHYROXINE SODIUM": 12, "POTASSIUM CHLORIDE": 30, "FUROSEMIDE": 162, "GLYBURIDE": 16, "CLONIDINE HCL": 43, "TEMAZEPAM": 41, "SPIRONOLACTONE": 50, "LOVASTATIN": 11, "LISINOPRIL": 44, "PANTOPRAZOLE SODIUM": 13, "CALCIUM ACETATE": 85, "NEXIUM": 44, "ZOLPIDEM TARTRATE": 41, "DIOVAN": 20, "OXYCODONE HCL": 51, "METOPROLOL SUCCINATE": 104, "RANITI

In [12]:
# read in data
raw_data = pd.read_json("../ETL/roam_prescription_based_prediction.jsonl",
                        lines=True,
                        orient='columns')
print(raw_data.shape)
raw_data.head()

(239930, 3)


,cms_prescription_counts,provider_variables,npi
0,"{'DOXAZOSIN MESYLATE': 26, 'MIDODRINE HCL': 12...","{'settlement_type': 'non-urban', 'generic_rx_c...",1295763035
1,"{'CEPHALEXIN': 23, 'AMOXICILLIN': 52, 'HYDROCO...","{'settlement_type': 'non-urban', 'generic_rx_c...",1992715205
2,"{'CEPHALEXIN': 28, 'AMOXICILLIN': 73, 'CLINDAM...","{'settlement_type': 'non-urban', 'generic_rx_c...",1578587630
3,{'AMOXICILLIN': 63},"{'settlement_type': 'non-urban', 'generic_rx_c...",1932278405
4,"{'PIOGLITAZONE HCL': 24, 'BENAZEPRIL HCL': 29,...","{'settlement_type': 'non-urban', 'generic_rx_c...",1437366804


We can see from above that we have nested values inside our cells. There are several options for extracting these values. In this kernel we will explore using list comprehensions and json_normalize.

# Extract Prescriber Data

## List Comprehension

In [13]:
%time provider = pd.DataFrame([md for md in raw_data.provider_variables])
provider.head()

CPU times: user 536 ms, sys: 49 ms, total: 585 ms
Wall time: 587 ms


,settlement_type,generic_rx_count,specialty,years_practicing,gender,region,brand_name_rx_count
0,non-urban,2287,Nephrology,7,M,South,384
1,non-urban,103,General Practice,7,M,South,0
2,non-urban,112,General Practice,7,M,Midwest,0
3,non-urban,63,General Practice,7,M,South,0
4,non-urban,1035,Nephrology,6,M,West,316


In [14]:
# add npi as index
provider['npi'] = raw_data.npi
provider.set_index('npi', inplace=True)
provider.head()

,settlement_type,generic_rx_count,specialty,years_practicing,gender,region,brand_name_rx_count
npi,,,,,,,
1295763035,non-urban,2287,Nephrology,7,M,South,384
1992715205,non-urban,103,General Practice,7,M,South,0
1578587630,non-urban,112,General Practice,7,M,Midwest,0
1932278405,non-urban,63,General Practice,7,M,South,0
1437366804,non-urban,1035,Nephrology,6,M,West,316


### JSON Normalize

In [15]:
%time provider = json_normalize(data=raw_data.provider_variables)
provider.head()

<timed exec>:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


CPU times: user 2.8 s, sys: 71.3 ms, total: 2.88 s
Wall time: 3.01 s


,settlement_type,generic_rx_count,specialty,years_practicing,gender,region,brand_name_rx_count
0,non-urban,2287,Nephrology,7,M,South,384
1,non-urban,103,General Practice,7,M,South,0
2,non-urban,112,General Practice,7,M,Midwest,0
3,non-urban,63,General Practice,7,M,South,0
4,non-urban,1035,Nephrology,6,M,West,316


### Extract Rx Data

#### List Comprehension

In [16]:
%time rx_counts = pd.DataFrame([rx for rx in raw_data.cms_prescription_counts])

CPU times: user 1min 57s, sys: 23.4 s, total: 2min 20s
Wall time: 2min 27s


In [17]:
print(rx_counts.shape)
rx_counts.head()

(239930, 2397)


,DOXAZOSIN MESYLATE,MIDODRINE HCL,MEGESTROL ACETATE,BENAZEPRIL HCL,METOLAZONE,NOVOLOG,DIAZEPAM,HYDRALAZINE HCL,SENSIPAR,LABETALOL HCL,...,HYOMAX-SL,CORTISPORIN,LOFIBRA,BIAXIN,BD INSULIN SYRINGE ULTRA-FINE,LUER-LOK SYRINGE,ZIRGAN,LOTRISONE,HEPARIN SODIUM-D5W,TESTRED
0,26.0,12.0,11.0,11.0,73.0,12.0,24.0,50.0,94.0,28.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,29.0,NaN,NaN,26.0,12.0,90.0,13.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
